# Additional Cleaning

Ziel: Überprüfen, ob es noch potenziell störende Sonderzeichen gibt und wie viele; wenn das viele sind, bestimmte Sonderzeichen ggf. ersetzen/Reden ausschließen. Wir nutzen dafür zumindest zum Teil die library clean-text, die vieles automatisiert.

In [ ]:
!pip install clean-text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.6 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171033 sha256=020ee1ccbcb9043f16bdee0d6fd77624ae258b7232eb253665da4cc46ac5c139
  Stored in directory: /root/.cache/pip/wheels/bd/22/e5/b69726d5e1a19795ecd3b3e7464b16c0f1d019aa94ff1c8578
Successfully built emoji


In [ ]:
import pandas as pd
import numpy as np
import os
from collections import Counter
from nltk import ngrams
from functools import reduce
import string
from cleantext import clean

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Daten laden (wichtig: damit man Daten aus einem geteilten Google Drive-Ordner laden kann, muss man für diesen einmal in seinem eigenen Drive eine Verknüpfung hinzufügen):

In [ ]:
path = '/content/drive/MyDrive/techlabs/Github/Daten/'
df = pd.read_parquet(path + "datatechlabs_2020_newclean.parquet")

Einmal kurz Daten anschauen:

In [ ]:
df.describe()

,Wahlperiode,Datum,Redner*in-ID,Redner*in Vorname,Redner*in Nachname,Rolle,Fraktion,Rede
count,38891,38891,38891,38891,38891,38891,38891,38891
unique,2,312,1020,509,928,122,6,38828
top,20,21.09.2023,11003231,Michael,Müller,keine,CDU/CSU,Ja.
freq,26151,226,471,784,520,33778,10083,26


In [ ]:
df.head()

,Wahlperiode,Datum,Redner*in-ID,Redner*in Vorname,Redner*in Nachname,Rolle,Fraktion,Rede
0,19,15.01.2020,11004699,Astrid,Damerow,keine,CDU/CSU,Frau Präsidentin! Verehrte Kolleginnen und Kol...
1,19,15.01.2020,11004393,Johann,Saathoff,keine,SPD,Sehr geehrte Frau Präsidentin! Liebe Kolleginn...
2,19,15.01.2020,11003706,Artur,Auernhammer,keine,CDU/CSU,Verehrte Frau Präsidentin! Liebe Kolleginnen u...
3,19,15.01.2020,11003604,Friedrich,Ostendorff,keine,Die Grünen,Sehr geehrte Frau Präsidentin! Liebe Kolleginn...
4,19,15.01.2020,11003740,Heidrun,Bluhm-Förster,keine,Die Linke,Frau Präsidentin! Verehrte Kolleginnen und Kol...


## Sonderzeichen

Erstmal schauen wir uns an, welche Sonderzeichen in den Reden noch vorkommen, und wie häufig:

In [ ]:
all_chars = "".join(df['Rede'].tolist())
char_frequencies = Counter(all_chars)

irregular_char_frequencies = Counter({char: freq for char, freq in char_frequencies.items() if char not in string.ascii_letters})

print(irregular_char_frequencies)

Counter({' ': 19414702, ',': 1338005, '.': 1043422, 'ü': 683023, 'ä': 550371, 'ö': 269925, '–': 175816, ':': 146241, 'ß': 122608, '0': 116790, '-': 114723, '!': 93930, '2': 77044, ';': 61455, '1': 56413, '\n': 55811, '?': 48359, '„': 36842, '“': 36810, '5': 27327, '3': 23832, '4': 18772, '9': 17415, 'Ü': 16520, '6': 15127, '7': 13821, '8': 13125, '\u202f': 11664, 'Ä': 9307, '/': 8923, 'Ö': 7375, '‑': 3396, '§': 2499, '…': 2275, '’': 1042, 'é': 627, ')': 275, 'ʼ': 206, 'à': 193, '&': 178, '‚': 168, '(': 152, '#': 146, 'ğ': 116, 'á': 115, 'è': 90, '+': 89, 'ć': 82, 'É': 61, 'ó': 58, 'í': 54, '*': 52, 'ç': 42, 'å': 25, 'ï': 24, 'č': 22, '"': 21, '[': 16, ']': 16, '_': 15, 'ś': 14, '%': 13, 'ô': 13, 'ł': 13, '‘': 12, 'ê': 10, 'ă': 10, "'": 10, '@': 10, 'ę': 10, 'İ': 10, 'ş': 9, '”': 8, 'ě': 7, 'λ': 6, 'ñ': 6, 'ž': 6, 'ā': 6, 'ē': 6, 'ã': 5, 'ń': 5, 'Č': 5, 'î': 5, 'š': 5, 'ė': 5, '‒': 4, '=': 4, '·': 4, 'Å': 3, 'ν': 3, 'ο': 3, 'Ş': 3, '\u2001': 3, '€': 2, '°': 2, 'À': 2, 'â': 2, 'ź': 2, 'ό

Folgende Sonderzeichen fallen auf:
- \n, also der Zeilenumbruch. Die würde ich einfach durch ein Leerzeichen ersetzen.
- Bei Satzzeichen (auch Anführungszeichen) würde ich erstmal davon ausgehen, dass die kein Problem sind
- Was ist mit Umlauten?
- \u202f ist ein sog. schmales, geschütztes Leerzeichen (d.h., wenn man dieses Leerzeichen setzt, gibt es keinen Zeilenumbruch). Ich denke, das können wir einfach durch ein Leerzeichen ersetzen.
- andere Unicode-Sachen wie \u2001, \u2002, \u2004 und \u2005 kommen nur selten vor, sind aber auch alle irgendeine Form von Leerzeichen und können dadurch ersetzt werden.
- Es macht wahrscheinlich Sinn, mal nachzuschauen, in welchen Reden /, §, …, @, &, #, +, *, %, •, =, ·, °, ², ⅓, ⅔, und in welchen Kontexten vorkommen.
- Was ist ´ für ein Zeichen? Kommt nur zweimal vor.
- Die anderen Sonderzeichen sind fast alle Buchstaben aus anderen Sprachen, ich würde vermuten dass die in anderssprachigen Namen vorkommen, die der Tokenizer eh nicht erkennt und durch einen Platzhalter ersetzt.


Wir schauen uns zunächst an, in welchen Kontexten die häufigsten Sonderzeichen verwendet werden:

In [ ]:
def find_sentences_with_weird_chars(text, weird_chars):
    sentences = text.split('.')
    sentences_with_weird_chars = []

    for sentence in sentences:
        has_weird_char = False  # Flag to track if a weird character is found
        for char in weird_chars:
            if char in sentence:
                has_weird_char = True
                break  # Exit inner loop if a weird character is found

        if has_weird_char:
            sentences_with_weird_chars.append(sentence)

    return sentences_with_weird_chars

In [ ]:
frequent_weird_chars = ['/', '§', '…']

weird_sentences1 = df['Rede'].apply(lambda x: find_sentences_with_weird_chars(x, frequent_weird_chars))
weird_sentences1 = weird_sentences1[weird_sentences1.apply(lambda x: len(x) > 0)] # remove empty entries

for i in range(len(weird_sentences1)):
  for j in range(len(weird_sentences1.iloc[i])):
    sentence = weird_sentences1.iloc[i][j]
    # Split the sentence into chunks of 80 characters
    chunks = [sentence[k:k+80] for k in range(0, len(sentence), 80)]
    # Print each chunk with a newline character
    for chunk in chunks:
        print(chunk)

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
 Stattdessen Ihre üblichen Tiraden und natürlich Ablehnung! Das Lieferkettengese
tz abzuschaffen, fordern wir schon seit 2022, Drucksache 20/4065
 Davon profitieren Mieter/-innen und Hausbesitzer/-innen, Gewerbetreibende und L
andwirtinnen und Landwirte
 Dieser Naturschutzstandard sichert für Betreiber/-innen Flexibilität und schaff
t gleichzeitig ökologischen Mehrwert, zum Beispiel durch Wanderkorridore für Tie
re
 Der Kollege Oster hat es gesagt: Die CDU/CSU-Fraktion wird am Freitag konkrete 
Aktionspläne dazu vorlegen
 Es gibt ja schon kleinere Initiativen, wenn ich beispielsweise an Frankfurt/Ode
r mit Słubice als Nachbarstadt denke
 Ich würde mir nämlich noch zwei zusätzliche Punkte wünschen, und zwar Punkt 17:
 Wir als CDU/CSU-Fraktion werden nicht Dinge im Wahlkampf propagieren, die der P
rävention entgegenstehen und die wir dann mit viel Geld für die Prävention wiede
r einfangen
 Ich schätze das wirklich, und ic

Was auffällt:
- Das Paragraphenzeichen § tritt erwartbar eigentlich immer dann auf, wenn ein Pagaraph aus irgendeinem Gesetzbuch zitiert wird.
- "/" tritt i.d.R. auf, wenn die Fraktionen "CDU/CSU" oder "Bündnis 90/Die Grünen" genannt werden, oder beim Gendern ("Mitberichterstatter/-innen")
- "…" wird häufig zu Beginn von Zitaten verwendet, das könnte man vmtl. einfach wegmachen.

Was ist mit den weniger häufigen Sonderzeichen?

In [ ]:
lessfrequent_weird_chars = ['@', '&', '#', '+', '*', '%', '•', '=', '·', '°', '²', '⅓', '⅔', '´', '¡']

weird_sentences2 = df['Rede'].apply(lambda x: find_sentences_with_weird_chars(x, lessfrequent_weird_chars))
weird_sentences2 = weird_sentences2[weird_sentences2.apply(lambda x: len(x) > 0)] # remove empty entries

for i in range(len(weird_sentences2)):
  for j in range(len(weird_sentences2.iloc[i])):
    sentence = weird_sentences2.iloc[i][j]
    # Split the sentence into chunks of 80 characters
    chunks = [sentence[k:k+80] for k in range(0, len(sentence), 80)]
    # Print each chunk with a newline character
    for chunk in chunks:
        print(chunk)

 Sie haben 1893 das Patent angemeldet und trugen sogar bis letztes Jahr das Wort
 „Diesel“ in ihrem Namen: MAN Diesel & Turbo
 Denken wir zum Beispiel an Erasmus und das Nachfolgeprogramm Erasmus+, EU-Förde
rprogramme für allgemeine und berufliche Bildung
 Landwirtschaftsministerin Klöckner hat ja so eine Internetkampagne mit Bildchen
 und Texten gestartet unter dem Titel #Dorfkinder, um das Image des ländlichen R
aums etwas aufzupeppen
 Was die Presseberichterstattung angeht: Der „Tagesspiegel Background“ sagt in d
er Rubrik „Energie & Klima“, dass das Gebäudeenergiegesetz besser ist, als viele
 glauben
 Diese sind beim Deutschen Roten Kreuz & Co auch bestens aufgehoben
 Israel ist auch Teil von „Erasmus+“, das heißt, junge Leute haben die Chance au
f einen Austausch
 Aber man muss sehen, dass die Hälfte davon an DB Netz und an DB Station & Servi
ce durch die Trassenentgelte und die Stationsgebühren geht
 Wir führen hier bereits Gespräche mit verschiedenen internationalen Förderorgan


- "\*" tritt z.B. bei 'LSBTIQ\*' auf
- "\#" tritt tatsächlich auf, wenn über Hashtags (Twitter etc.) geredet wird
- "\&" kommt häufig in Namen vor, z.B. MAN Diesel & Turbo
- "+" scheint i.d.R. Erasmus+ zu sein, kommt aber auch manchmal in Namen vor (da würde ich nichts dran machen)
- "\*" wird auch manchmal zum Gendern verwendet.
- "\%" wird anscheinend wirklich i.d.R. als Prozentzeichen verwendet (wie erwartet/erhofft)

Was mit den seltenen weirden Sonderzeichen ist, sieht man immer noch nicht so gut. Deswegen noch mal einzeln für die:


In [ ]:
rare_weird_chars = ['@',  '•', '=', '·', '°', '²', '⅓', '⅔', '´', '¡']

weird_sentences3 = df['Rede'].apply(lambda x: find_sentences_with_weird_chars(x, rare_weird_chars))
weird_sentences2 = weird_sentences3[weird_sentences3.apply(lambda x: len(x) > 0)] # remove empty entries

for i in range(len(weird_sentences3)):
  for j in range(len(weird_sentences3.iloc[i])):
    sentence = weird_sentences3.iloc[i][j]
    # Split the sentence into chunks of 80 characters
    chunks = [sentence[k:k+80] for k in range(0, len(sentence), 80)]
    # Print each chunk with a newline character
    for chunk in chunks:
        print(i, j, chunk)

4769 0  Dabei sind Wirtschaft und Gesellschaft vielfach längst weiter als Sie: Ich nenn
4769 0 e Fridays/Parents/Scientists for Future, Appelle aus der Wissenschaft, das Weltw
4769 0 irtschaftsforum in Davos, Stiftung 2° oder jetzt gerade die Chefs von 150 intern
4769 0 ationalen Unternehmen wie Google, Apple, Deutsche Bank, die eine deutliche Senku
4769 0 ng des CO Und sie haben recht! Brände überall auf der Welt – der Regenwald, Kali
4769 0 fornien, die Tundra –, Überschwemmungen in Afrika, Dürre in Brandenburg, Waldste
4769 0 rben, Hitze in deutschen Städten – alles bei nur 1 Grad durchschnittlicher Erwär
4769 0 mung
7193 0  Sollte Ihre Hypothese stimmen – für die es keinen einzigen wissenschaftlichen B
7193 0 eweis gibt – und verschwände Deutschland in Jahresfrist komplett von der Klimala
7193 0 ndkarte, würde sich die globale Mitteltemperatur um ganze 0,000653° Celsius weni
7193 0 ger erhöhen
7358 0  Aber Sie müssen ja eines sehen: Wir haben am Anfang des Jahres, vor den Pandemi
7

Es gibt aus meiner Sicht zwei Zeichen, die wir vorher ersetzen können: Paragrafen- und Prozentzeichen. Die würde ich nicht prinzipiell löschen, sondern durch das jeweiligen Wort ersetzen. Das machen wir jetzt als Erstes:

In [ ]:
def clean_specchars(speech):
  speech.replace(' %', ' Prozent')
  speech.replace(' §', ' Paragraf')
  return(speech)

In [ ]:
df['Rede'] = df['Rede'].apply(clean_specchars)

Dann nutzen wir clean aus der cleantext library, um die Daten grundlegend zu bereinigen:

In [ ]:
def clean_customized(speech):
  speech = clean(speech,
                 fix_unicode=True,               # fix various unicode errors
                 to_ascii=True,                  # transliterate to closest ASCII representation
                 lower=False,                     # lowercase text
                 no_line_breaks=True,           # fully strip line breaks as opposed to only normalizing them
                 no_urls=False,                  # replace all URLs with a special token
                 no_emails=False,                # replace all email addresses with a special token
                 no_phone_numbers=False,         # replace all phone numbers with a special token
                 no_numbers=False,               # replace all numbers with a special token
                 no_digits=False,                # replace all digits with a special token
                 no_currency_symbols=False,      # replace all currency symbols with a special token
                 no_punct=False,                 # remove punctuations
                 replace_with_punct="",          # instead of removing punctuations you may replace them
                 replace_with_url="<URL>",
                 replace_with_email="<EMAIL>",
                 replace_with_phone_number="<PHONE>",
                 replace_with_number="<NUMBER>",
                 replace_with_digit="0",
                 replace_with_currency_symbol="<CUR>",
                 lang = "de")
  return(speech)

In [ ]:
df['Rede'] = df['Rede'].apply(clean_customized)

In [ ]:
# Eine Rede anschauen:
df.iloc[16]['Rede']

'Lieber Kollege Höferlin, vielen Dank für diese Frage. Wenn es Sicherheitslücken in anderen europäischen Ländern gibt, dann sollten wir uns doch gemeinsam bemühen das ist meine Meinung , diese zu schließen, statt sie in Deutschland zu übernehmen. Es kann doch nicht unser politisches Handeln sein, dass wir uns daran ein Beispiel nehmen. Das ist auf jeden Fall nicht unser Kurs; das kann ich Ihnen sagen. Ich habe es Ihnen gesagt: Auch die Inhaber von Privatpilotlizenzen haben Zugänge zu allen sicherheitsrelevanten Bereichen. Das, was Sie fordern, ist im Sinne einer Gleichbehandlung völlig absurd, und es ist auch nicht zu Ende gedacht. Deshalb lehnen wir den Antrag der FDP entschieden ab, freuen uns aber trotzdem auf die weiteren Beratungen mit Ihnen. Vielen Dank.'

In [ ]:
# Noch mal schauen, welche Sonderzeichen jetzt  noch drin sind:
all_chars2 = "".join(df['Rede'].tolist())
char_frequencies2 = Counter(all_chars2)

irregular_char_frequencies2 = Counter({char: freq for char, freq in char_frequencies2.items() if char not in string.ascii_letters})

print(irregular_char_frequencies2)

Counter({' ': 18223901, ',': 1338005, '.': 1043422, 'ü': 683023, 'ä': 550371, 'ö': 269925, ':': 146241, 'ß': 122608, '0': 116790, '-': 114723, '!': 93930, '2': 77044, '"': 73681, ';': 61455, '1': 56413, '?': 48359, '5': 27327, '3': 23832, '4': 18772, '9': 17415, 'Ü': 16520, '6': 15127, '7': 13821, '8': 13125, 'Ä': 9307, '/': 8923, 'Ö': 7375, "'": 1441, ')': 275, '&': 178, '(': 152, '#': 146, '+': 89, '*': 52, '[': 16, ']': 16, '_': 15, '%': 13, '@': 10, '=': 4})


Da ist nichts Wildes mehr dabei. Es bleiben noch ein paar Fragen offen, z.B.: Was ist mit Umlauten? Wie geht der Tokenizer tatsächlich mit anderen Buchstaben um, oder mit den Sonderzeichen, die wir nicht ersetzt haben? Das schaue ich mir mal in einem separaten Skript an.

In [ ]:
df.to_parquet(path + 'datatechlabs_2020_newclean_cleantext.parquet')